In [ ]:
from regions import CircleSkyRegion
import operator

And more imports from `gammapy`:

In [ ]:
from gammapy.data import Observation, observatory_locations
from gammapy.datasets import SpectrumDataset, SpectrumDatasetOnOff
from gammapy.irf import load_cta_irfs
from gammapy.makers import SpectrumDatasetMaker
from gammapy.maps import MapAxis, RegionGeom
from gammapy.modeling import Fit
from gammapy.modeling import models 
from gammapy.estimators import FluxPointsEstimator

## CTA sensitivity for a point-like IRF 

[gammapy-tutorial](https://docs.gammapy.org/1.2/tutorials/analysis-1d/cta_sensitivity.html)</br>

[CTAO's expected "Alpha Configuration" performance](https://www.cta-observatory.org/science/ctao-performance/)

[CTAO Instrument Response Functions - prod5 version v0.1](https://zenodo.org/records/5499840#.YUya5WYzbUI)

[IRFs - gamma-astro-data-formats](https://gamma-astro-data-formats.readthedocs.io/en/v0.3/irfs/index.html)

In [1]:
%matplotlib inline

In [19]:
import matplotlib.pyplot as plt

import numpy as np
import astropy.units as u

from astropy.coordinates import Angle, SkyCoord

from feupy.utils.string_handling import name_to_txt
from feupy.utils.io import mkdir_sub_directory
from feupy.utils.table import write_tables_fits
from feupy.utils.table import write_tables_csv

from feupy.utils.geometry import (
    create_energy_axis, 
    create_pointing, 
    create_pointing_position, 
    create_region_geometry,
    define_on_region,
)

from feupy.analysis.config import AnalysisConfig
from feupy.analysis.core import Analysis
from feupy.utils.observation import get_obs_label
from feupy.cta.irfs import Irfs

from gammapy.maps import MapAxis

from gammapy.datasets import SpectrumDataset, SpectrumDatasetOnOff, Datasets
from gammapy.makers import SpectrumDatasetMaker, SafeMaskMaker
from gammapy.estimators import FluxPoints, SensitivityEstimator

from gammapy.data import Observation

In [3]:
from config import irfs_label_txt

In [ ]:
from astropy.visualization import quantity_support
import matplotlib.pyplot as plt
from gammapy.irf import (
    Background3D,
    EffectiveAreaTable2D,
    EnergyDependentMultiGaussPSF,
    EnergyDispersion2D,
)
from gammapy.irf.io import COMMON_IRF_HEADERS, IRF_DL3_HDU_SPECIFICATION
from gammapy.makers.utils import (
    make_edisp_kernel_map,
    make_map_exposure_true_energy,
    make_psf_map,
)
from gammapy.maps import MapAxis, WcsGeom

In [ ]:
irfs_groups =[
    ['South', 'South-SSTSubArray','South-MSTSubArray','North','North-MSTSubArray', 'North-LSTSubArray'], 
    ['AverageAz', 'SouthAz', 'NorthAz'], 
    ['20deg','40deg','60deg'], 
    ['0.5h', '5h', '50h']
]
IRFS_OPTS, IRFS, IRFS_LABELS, LOCATION = Irfs.get_irf_groups(irfs_groups)

In [4]:
config = AnalysisConfig()

In [5]:
source_name = 'NGC 1275'

position = SkyCoord.from_name(source_name)
print(f'{position.ra.deg:.2f}, {position.dec.deg:.2f}')
print(f'{position.galactic.l.deg:.2f}, {position.galactic.b.deg:.2f}')

49.95, 41.51
150.58, -13.26


In [6]:
outdir = f"./{name_to_txt(source_name)}"
outdir_path = mkdir_sub_directory(outdir)
datasets_path = mkdir_sub_directory(outdir, 'datasets')[1]
figures_path = mkdir_sub_directory(outdir, 'figures')[1]
sensitivity_path = mkdir_sub_directory(outdir, 'sensitivity')[1]
data_path = mkdir_sub_directory(outdir, 'data')[1]

config.general.outdir = outdir
config.general.datasets_file = f'{datasets_path}/datasets.yaml'
config.general.models_file = f'{datasets_path}/models.yaml'

Directory 'NGC_1275' created
Directory 'NGC_1275/datasets' created
Directory 'NGC_1275/figures' created
Directory 'NGC_1275/sensitivity' created
Directory 'NGC_1275/data' created


In [7]:
analysis = Analysis(config)

Setting logging config: {'level': 'INFO', 'filename': None, 'filemode': None, 'format': None, 'datefmt': None}


In [8]:
# analysis.read_datasets()
analysis.datasets =Datasets()

In [9]:
# Define simulation parameters parameters
livetime = 5. * u.h

pointing_angle = 0*u.deg
offset = 0.5*u.deg

on_region_radius = Angle("0.11 deg")

gamma_min = 10
n_sigma = 3 
bkg_syst_fraction = 0.10

selection = ["edisp", "background", "exposure"]

containment = 0.68

acceptance = 1
acceptance_off = 20

In [10]:
pointing_position = create_pointing_position(position, pointing_angle, offset)
pointing = create_pointing(pointing_position)
print(f"{pointing}\n")


on_region = define_on_region(center=position, radius=on_region_radius)
print(f"{on_region}\n")

e_edges_min = 1.0e-01*u.TeV
e_edges_max = 3.2e+01*u.TeV
nbin_edges = 12
config.datasets.geom.axes.energy.min = e_edges_min
config.datasets.geom.axes.energy.max = e_edges_max
config.datasets.geom.axes.energy.nbins = nbin_edges
config.datasets.geom.axes.energy.name = 'energy'
energy_settings = config.datasets.geom.axes.energy

e_edges_min = 3.2e-02*u.TeV
e_edges_max = 1.0e+02*u.TeV
nbin_edges = 15
config.datasets.geom.axes.energy_true.min =  e_edges_min
config.datasets.geom.axes.energy_true.max = e_edges_max
config.datasets.geom.axes.energy_true.nbins = nbin_edges
config.datasets.geom.axes.energy_true.name = 'energy_true'
energy_true_settings = config.datasets.geom.axes.energy_true

energy_axis = create_energy_axis(
    energy_settings.min, 
    energy_settings.max, 
    energy_settings.nbins, 
    per_decade=True, 
    name=energy_settings.name
)

geom = create_region_geometry(on_region, axes=[energy_axis])
print(geom)
print(energy_axis)

energy_axis_true = create_energy_axis(
    energy_true_settings.min, 
    energy_true_settings.max, 
    energy_true_settings.nbins, 
    per_decade=True, 
    name=energy_true_settings.name
)


empty_dataset = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true, name='empty_dataset')
# analysis.datasets = Datasets(empty_dataset)
print(empty_dataset)

FixedPointingInfo:

mode:        PointingMode.POINTING
coordinates: <SkyCoord (ICRS): (ra, dec) in deg
    (49.95066663, 42.0116969)>

Region: CircleSkyRegion
center: <SkyCoord (ICRS): (ra, dec) in deg
    (49.95066663, 41.5116969)>
radius: 0.11 deg

RegionGeom

	region     : CircleSkyRegion
	axes       : ['lon', 'lat', 'energy']
	shape      : (1, 1, 31)
	ndim       : 3
	frame      : icrs
	center     : 50.0 deg, 41.5 deg

MapAxis

	name       : energy    
	unit       : 'TeV'     
	nbins      : 31        
	node type  : edges     
	edges min  : 1.0e-01 TeV
	edges max  : 3.2e+01 TeV
	interp     : log       

SpectrumDataset
---------------

  Name                            : empty_dataset 

  Total counts                    : 0 
  Total background counts         : 0.00
  Total excess counts             : 0.00

  Predicted counts                : 0.00
  Predicted background counts     : 0.00
  Predicted excess counts         : nan

  Exposure min                    : 0.00e+00 m2 s
  Expos

In [11]:
spectrum_maker = SpectrumDatasetMaker(selection=selection)
sensitivity_estimator = SensitivityEstimator(
    gamma_min=gamma_min, n_sigma=n_sigma, bkg_syst_fraction=bkg_syst_fraction
)
sensitivity_estimator1 = SensitivityEstimator(
    gamma_min=gamma_min, n_sigma=n_sigma, bkg_syst_fraction=bkg_syst_fraction
)

In [12]:
livetime = 25 * u.h

In [ ]:
IRFS_OPTS

In [ ]:
idx=0
irfs_opts, irfs, irfs_label, location = IRFS_OPTS[idx], IRFS[idx], IRFS_LABELS[idx], LOCATION[idx]

In [13]:
irfs_config =[
    ['South', 'South-SSTSubArray','South-MSTSubArray','North','North-MSTSubArray', 'North-LSTSubArray'], 
    ['AverageAz'], 
    ['20deg','40deg','60deg'], 
    ['50h']
]
irfs_opt, irfs, irfs_label, location = Irfs.get_irf_groups(irfs_config)

In [15]:
irfs_opt = ['North', 'AverageAz', '20deg', '50h']
irfs = Irfs.get_irfs(irfs_opt)
irfs_label = Irfs.get_irfs_label(irfs_opt)
location = Irfs.get_obs_loc(irfs_opt)

In [ ]:
aeff = irfs['aeff']
psf = irfs['psf']
edisp = irfs['edisp']
bkg = irfs['bkg']

In [ ]:
aeff = irfs['aeff']
print(aeff)

In [ ]:
# To see the IRF axes binning, eg, offset
print(aeff.axes["offset"])

# To get the IRF data
print(aeff.data)

# the aeff is evaluated at a given energy and offset
print(aeff.evaluate(energy_true=energy_axis_true.bounds, offset=[0.2, 2.5] * u.deg))


# The peek method gives a quick look into the IRF
aeff.peek()

In [ ]:
print(bkg)

In [ ]:
print(bkg.fov_alignment)

In [ ]:
print(bkg.interp_kwargs)

In [ ]:
# Evaluate background
# Note that evaluate functions support  numpy-style broadcasting
energy = energy_axis_true.edges
fov_lon = [1, 2] * u.deg
fov_lat = [1, 2] * u.deg
ev = bkg.evaluate(
    energy=energy.reshape(-1, 1, 1),
    fov_lat=fov_lat.reshape(1, -1, 1),
    fov_lon=fov_lon.reshape(1, 1, -1),
)
print(ev)


In [ ]:
print(
    "Interpolation scheme for energy axis is: ",
    bkg.axes["energy"].interp,
    "and for the fov_lon axis is: ",
    bkg.axes["fov_lon"].interp,
)

In [ ]:
bkg.plot_at_energy(energy=energy_axis_true.bounds)

In [ ]:
# Evaluate energy dispersion
ev = edisp.evaluate(energy_true=1 * u.TeV, offset=[0, 1] * u.deg, migra=[1, 1.2])
print(ev)

edisp.peek()

print(psf)

In [ ]:
edisp.plot_migration()

In [ ]:
print(psf.axes.names)

# To get the containment radius for a fixed fraction at a given position
print(
    psf.containment_radius(
        fraction=0.68, energy_true=1.0 * u.TeV, offset=[0.2, 4.0] * u.deg
    )
)

# Alternatively, to get the containment fraction for at a given position
print(
    psf.containment(
        rad=0.05 * u.deg, energy_true=1.0 * u.TeV, offset=[0.2, 4.0] * u.deg
    )
)

In [ ]:
psf.plot_containment_radius_vs_energy()

In [ ]:
psf.plot_psf_vs_rad()

In [ ]:
psf.plot_containment_radius()

0.11

In [32]:
obs = Observation.create(
    pointing=pointing, irfs=irfs, livetime=livetime, location=location
)
print(obs)

dataset = spectrum_maker.run(empty_dataset, obs)
print(dataset)

# correct exposure
print('correct exposure')
dataset.exposure *= containment
print(dataset)

# correct background estimation
print('correct background estimation')
on_radii = obs.psf.containment_radius(
    energy_true=energy_axis.center, 
    offset=offset, 
    fraction=containment
)
factor = (1 - np.cos(on_radii)) / (1 - np.cos(geom.region.radius))
dataset.background *= factor.value.reshape((-1, 1, 1))
print(dataset)

print('create SpectrumDatasetOnOff')
dataset_on_off = SpectrumDatasetOnOff.from_spectrum_dataset(
    dataset=dataset, acceptance=acceptance, acceptance_off=acceptance_off
)
print(dataset_on_off)

sensitivity_table = sensitivity_estimator.run(dataset_on_off)
sensitivity_table.meta['source'] = source_name
sensitivity_table.meta["offset"] = offset.to_string()
sensitivity_table.meta["on_region_radius"] = f'{on_region_radius.deg}deg'
sensitivity_table.meta["livetime"] = livetime.to_string()    
sensitivity_table.meta["site"] = irfs_opt[0]
sensitivity_table.meta["azimuth-averaged "] = irfs_opt[1]
sensitivity_table.meta["zenith-angle"] = u.Quantity(irfs_opt[2]).to_string()
sensitivity_table.meta["obs_time"] = u.Quantity(irfs_opt[3]).to_string()
sensitivity_table.meta['irfs_label'] = irfs_label
sensitivity_table.meta['irfs_config'] = irfs_opt
sensitivity_table["on_radii"] = on_radii
sensitivity_table["on_radii"].format = '.3e'
write_tables_csv(
    sensitivity_table, sensitivity_path, f'sens-{irfs_label_txt(irfs_opt)}')
print(sensitivity_table)
print(dataset_on_off)


dataset_on_off1 = dataset_on_off.to_image()

sensitivity_table1 = sensitivity_estimator1.run(dataset_on_off1)
print(sensitivity_table1)

# To get the integral flux, we convert to a `FluxPoints` object that does the conversion
# internally
flux_points = FluxPoints.from_table(
    sensitivity_table1, sed_type="e2dnde", reference_model=sensitivity_estimator1.spectrum
)
int_sens = np.squeeze(flux_points.flux.quantity)
print(
    f"Integral sensitivity in {livetime:.2f} above {e_edges_min:.2e} "
    f"is {int_sens:.2e}"
)

Observation

	obs id            : 0 
 	tstart            : 51544.00
	tstop             : 51545.04
	duration          : 90000.00 s
	pointing (icrs)   : 50.0 deg, 42.0 deg

	deadtime fraction : 0.0%



Table column name energy will be deprecated by e_ref since v1.2


SpectrumDataset
---------------

  Name                            : empty_dataset 

  Total counts                    : 0 
  Total background counts         : 2287.07
  Total excess counts             : -2287.07

  Predicted counts                : 2287.07
  Predicted background counts     : 2287.07
  Predicted excess counts         : nan

  Exposure min                    : 1.73e+09 m2 s
  Exposure max                    : 8.88e+10 m2 s

  Number of total bins            : 31 
  Number of fit bins              : 31 

  Fit statistic type              : cash
  Fit statistic value (-2 log(L)) : nan

  Number of models                : 0 
  Number of parameters            : 0
  Number of free parameters       : 0


correct exposure
SpectrumDataset
---------------

  Name                            : empty_dataset 

  Total counts                    : 0 
  Total background counts         : 2287.07
  Total excess counts             : -2287.07

  Predicted counts                : 2287.07
 

Table column name energy will be deprecated by e_ref since v1.2


 energy  e_ref  e_min e_max     e2dnde    excess background criterion
  TeV     TeV    TeV   TeV  erg / (s cm2)                            
------- ------- ----- ----- ------------- ------ ---------- ---------
1.78885 1.78885   0.1    32   2.49458e-13 218.95     2189.5       bkg
Integral sensitivity in 25.00 h above 3.20e-02 TeV is 1.55e-12 1 / (s cm2)


In [ ]:
idx=0

for idx_irfs, irfs in enumerate(irfs_opts[0:1]):
    obs_location = obs_locations[idx_irfs]
    irfs_label = f"{irfs_labels[idx_irfs]}"
        
    print(f"Simulation: obs-{idx_irfs}, irfs = {irfs_label}, livetime: {livetime}\n")
    
    obs_label_full = get_obs_label(irfs_groups[idx_irfs], offset, on_region_radius, livetime, txt=True)
    obs_label = get_obs_label(irfs_groups[idx_irfs], offset, on_region_radius, livetime)


    obs = Observation.create(
        pointing=pointing, irfs=irfs, livetime=livetime, location=obs_location
    )
    print(obs)

    dataset = spectrum_maker.run(empty_dataset, obs)
    print(dataset)

    # correct exposure
    dataset.exposure *= containment

    # correct background estimation
    on_radii = obs.psf.containment_radius(
        energy_true=energy_axis.center, 
        offset=offset, 
        fraction=containment
    )
    
    factor = (1 - np.cos(on_radii)) / (1 - np.cos(geom.region.radius))
    dataset.background *= factor.value.reshape((-1, 1, 1))
    print(dataset)

    dataset_on_off = SpectrumDatasetOnOff.from_spectrum_dataset(
        dataset=dataset, acceptance=acceptance, acceptance_off=acceptance_off
    )
    analysis.datasets.append(dataset_on_off.copy(name=f'onoff {irfs_label}'))
    print(dataset_on_off)

    sensitivity_table = sensitivity_estimator.run(dataset_on_off)
    
    sensitivity_table.meta['source'] = source_name
    sensitivity_table.meta["obs_id"] = f"obs-{idx}"
    sensitivity_table.meta["offset"] = offset.to_string()
    sensitivity_table.meta["on_region_radius"] = on_region_radius.to_string()
    sensitivity_table.meta["livetime"] = livetime.to_string()
    sensitivity_table.meta["irf_id"] = f"irf-{idx_irfs}"
    
    sensitivity_table.meta["site"] = irfs_groups[idx_irfs][0]
    sensitivity_table.meta["azimuth-averaged "] = irfs_groups[idx_irfs][1]
    sensitivity_table.meta["zenith-angle"] = u.Quantity(irfs_groups[idx_irfs][2]).to_string()
    sensitivity_table.meta["obs_time"] = u.Quantity(irfs_groups[idx_irfs][3]).to_string()
    sensitivity_table.meta['irfs_label'] = irfs_label
    sensitivity_table.meta['irfs_config'] = irfs_groups[idx_irfs]
    sensitivity_table.meta['obs_label_full'] = obs_label_full
    sensitivity_table["on_radii"] = on_radii
    print(sensitivity_table)    
    file_path = f'{sensitivity_path}/sensitivity-{irfs_label_txt(irfs_groups[idx_irfs])}'
    sensitivity_table.write(f'{file_path}.ecsv', format='ascii.ecsv', overwrite=True)
    dataset_sens = flux_points_dataset_from_table(sensitivity_table, name = f"sens {obs_label}" )
    analysis.datasets.append(dataset_sens)
        
    print(dataset_on_off)

    dataset_on_off1 = analysis.datasets[f'onoff {irfs_label}'].copy(name=f'onoff1 {irfs_label}').to_image()

    sensitivity_table1 = sensitivity_estimator1.run(dataset_on_off1)
    print(sensitivity_table1)

    # To get the integral flux, we convert to a `FluxPoints` object that does the conversion
    # internally
    flux_points = FluxPoints.from_table(
        sensitivity_table1, sed_type="e2dnde", reference_model=sensitivity_estimator1.spectrum
    )
    int_sens = np.squeeze(flux_points.flux.quantity)
    print(
        f"Integral sensitivity in {livetime:.2f} above {e_edges_min:.2e} "
        f"is {int_sens:.2e}"
    )

    sensitivity_table.meta["int_sens"] = int_sens.to_string()
    idx+=1

In [ ]:
from feupy.cta.irfs import Irfs
from feupy.utils.geometry import *

In [ ]:
# irfs = load_cta_irfs(
#     "./Prod5-North-20deg-AverageAz-4LSTs09MSTs.18000s-v0.1.fits.gz"
# )

# Loading IRFs
irfs_opts =['South', 'AverageAz', '40deg', '50h']
irfs = Irfs.get_irfs(irfs_opts)
obs_location = Irfs.get_obs_loc(irfs_opts)
irfs_label = Irfs.get_irfs_label(irfs_opts)

In [ ]:
obs = Observation.create(
    pointing=pointing,
    livetime=livetime,
    irfs=irfs,
    location=obs_location,
)
print(obs)

#### Peek at the IRFs

We check the effective area, PSF, and energy dispersion matrix.

In [ ]:
irfs["aeff"].peek()

In [ ]:
irfs["edisp"].peek()

In [ ]:
irfs["psf"].peek()

### Simulate a spectra

Now we have to follow some steps to simulate the On/Off data set.


In [ ]:
# Make the SpectrumDataset
geom = RegionGeom.create(region=on_region, axes=[energy_axis])

dataset_empty = SpectrumDataset.create(
    geom=geom, energy_axis_true=energy_axis_true, name="obs-0"
)
maker = SpectrumDatasetMaker(selection=["exposure", "edisp", "background"])

dataset = maker.run(dataset_empty, obs)

# Set the model on the dataset, and fake
dataset.models = model_no_alps
dataset.fake(random_state=42)
print(dataset)

In [ ]:
dataset.peek()

You can see that background counts are now simulated.

### On-Off analysis

To do an on off spectral analysis, which is the usual science case, the
standard would be to use `SpectrumDatasetOnOff`, which uses the
acceptance to fake off-counts.
The `acceptance_off` is essentially the parameter $\alpha^{-1}$. 

In [ ]:
dataset_on_off = SpectrumDatasetOnOff.from_spectrum_dataset(
    dataset=dataset, acceptance=1, acceptance_off=5
)
dataset_on_off.fake(npred_background=dataset.npred_background(), random_state=42)
print(dataset_on_off)

In [ ]:
dataset_on_off.peek()

### Look at the statistics of the dataset

Basic statistics of the dataset like On, Off, excess counds, and the source significance are available from the `info_dict`.

In [ ]:
dataset_on_off.info_dict()

### Perform the fit

For our baseline model, without ALPs, we fit the observation with the same model that we used to simulate the data set, i.e., our intrinsic model with EBL absorption. 

In [ ]:
help(Fit)

In [ ]:
# init the fit
fit = Fit(optimize_opts={"print_level":2})

In [ ]:
%%time
result_no_alps = fit.run(dataset_on_off)

In [ ]:
# check the best-fit parameters
result_no_alps.models[0].spectral_model.parameters.to_table()

The Cash statistic, $C = -2\ln\mathcal{L}$, is stored in `results_no_alps.total_stat`:

In [ ]:
result_no_alps.total_stat

One can easily plot the excess counts and the predicted signal counts:

In [ ]:
plt.figure()
ax_spectrum, ax_residuals = dataset_on_off.plot_fit()
dataset_on_off.plot_masks(ax=ax_spectrum)

Before performing the fit with ALPs, we generate flux points, so that we can plot the SED.

In [ ]:
fpe = FluxPointsEstimator(
    energy_edges=energy_axis.edges, reoptimize=False, selection_optional=["errn-errp", "ul"], source="ngc1275"
)
flux_points = fpe.run(datasets=dataset_on_off)

Plot the flux points, define the energy range to flux points with a $\mathrm{TS}$ values larger than 1

In [ ]:
# TS values of flux points
ts_values = np.squeeze(flux_points.ts.data)

# mask 
mask_ts = ts_values > 1.

# energy range for plotting
plot_range = flux_points.energy_axis.edges[:-1][mask_ts][0] * 0.9, flux_points.energy_axis.edges[1:][mask_ts][-1]
print(plot_range)

# plot the model
ax = result_no_alps.models[0].spectral_model.plot(energy_bounds=plot_range, energy_power=2, ls="-")

# plot the butterfly, i.e., the model error
result_no_alps.models[0].spectral_model.plot_error(energy_bounds=plot_range, energy_power=2, ax=ax)

# plot the flux points
flux_points.plot(ax=ax, sed_type="e2dnde", color="darkorange")

# limit the energy range
plt.xlim(plot_range)

## Fit with Axion-like particles 

As a last step, we perform a fit with axion-like particles. For that, we loop through the 
$B$ field realizations. We first fit and plot one example. Then we perform the loop.

First we define a new compound model that includes the ALP effect. 

In [ ]:
# use the first B field realization
alps = models.TemplateSpectralModel(ml.EGeV * u.GeV, pgg[0])

# and define the compound model
model_obs_with_alps = models.CompoundSpectralModel(model_intrinsic, alps, operator=operator.mul)

model_w_alps = models.SkyModel(spectral_model=model_obs_with_alps, name="ngc1275")

Let's plot the different models

In [ ]:
ax = model_intrinsic.plot(energy_bounds=[0.1, 3.] * u.TeV, energy_power=2, ls='--')
_ = model_input.plot(energy_bounds=[0.1, 3.] * u.TeV, energy_power=2)
_ = model_obs_with_alps.plot(energy_bounds=[0.1, 3.] * u.TeV, energy_power=2)

Then we set the model and fit it to data

In [ ]:
dataset_on_off.models = model_w_alps

In [ ]:
result_w_alps = fit.run(dataset_on_off)

In [ ]:
# check the best-fit parameters
result_w_alps.models[0].spectral_model.parameters.to_table()

In [ ]:
result_w_alps.total_stat

#### Questions:
- What do you notice for the best-fit parameters?
- Can you plot the flux points, the best fit without ALPs and the best-fit models with ALPs?

In [ ]:
# plot the model w/o ALPs
ax = result_no_alps.models[0].spectral_model.plot(energy_bounds=plot_range, energy_power=2, ls="-")
result_no_alps.models[0].spectral_model.plot_error(energy_bounds=plot_range, energy_power=2, ax=ax)

# plot the model w/ ALPs
ax = result_w_alps.models[0].spectral_model.plot(energy_bounds=plot_range, energy_power=2, ls="-")
result_w_alps.models[0].spectral_model.plot_error(energy_bounds=plot_range, energy_power=2, ax=ax)

# plot the flux points
flux_points.plot(ax=ax, sed_type="e2dnde", color="k")

# limit the energy range
plt.xlim(plot_range)

Lastly, we loop through the magnetic field realizations and perform the fit in each step. 
We save the $C$ stat value and plot the difference, $\Delta C = C_\mathrm{w/~ALP} - C_\mathrm{no~ALP}$.

In [ ]:
%%time

tot_stat = []

fit = Fit(optimize_opts={"print_level":1})  # reduce print level

for i, p in enumerate(pgg):
    
    # model
    alps = models.TemplateSpectralModel(ml.EGeV * u.GeV, pgg[i])
    model_obs_with_alps = models.CompoundSpectralModel(model_intrinsic, alps, operator=operator.mul)
    model_w_alps = models.SkyModel(spectral_model=model_obs_with_alps, name="ngc1275")
    
    # set the model of the dataset
    dataset_on_off.models = model_w_alps
    
    # fit 
    result_w_alps = fit.run(dataset_on_off)
    
    # save the total stat value
    tot_stat.append(result_w_alps.total_stat)

In [ ]:
# convert to np.array
tot_stat = np.array(tot_stat)

In [ ]:
# plot the histogram
llr = tot_stat - result_no_alps.total_stat  # the log likelihood ratio values
bins = np.linspace(0, llr.max(), 20)  # bin edges for histogram
plt.hist(llr, bins=bins)
plt.xlabel("$\Delta C = C_\mathrm{w/~ALP} - C_\mathrm{no~ALP}$")

# and the fifth quantile
print(int(len(tot_stat) * 0.05))
tot_stat_q5 = np.sort(tot_stat)[int(len(tot_stat) * 0.05)]

plt.axvline(tot_stat_q5 - result_no_alps.total_stat, ls='--', color='r')
print(tot_stat_q5 - result_no_alps.total_stat)

### Questions:
- How do interpret this histogram?
- Which B field realization should we use to decide if the ALP hypothesis is preferred or not?
- How do the results change when you change the observation live time?
- How do the results change when you change the intrinsic model to a power law?
- How do the results change when you change the ALP mass or photon-ALP coupling?